In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import scale 

Veri setimizi kullanmak üzere başka bir değişkene kopyaladık.

In [ ]:
startups = pd.read_csv("../input/bakircayd2do-tabletdataset/tablet.csv")
df = startups.copy()

Veri çerçevesinin ilk 5 gözlemini görüntüleyelim.

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

Yukarıda görüldüğü üzere: Standart Sapma değeri en yüksek olan RAM'dir. En küçük olan ise Kalinlik'dir. Buna göre Varyansı Yorumlayalım.


Veri çerçevesinin kaç öznitelik ve kaç gözlemleden oluştuğu bilgisine bakalım.

In [ ]:
df.shape

Verilerimizin niteliklerinin Data Type'larını görelim.

In [ ]:
df.dtypes

In [ ]:
df.tail()

Veri gözlemimizdeki eksik değerleri görelim.

In [ ]:
df.count()

Veri setimizdeki eksik değerlerin nerede bulunduğunu görelim


In [ ]:
df.isnull().sum()

Eksik değelerin kaç adet olduğunu görelim

In [ ]:
df.isnull().sum().sum()

Veri setimizin 'Standart Sapmasını','Ortalamasını','Medyan değerini' inceleyelim.

In [ ]:
df.std()

In [ ]:
df.mean()

In [ ]:
df.median()

Korelasyon Haritasini çizdirelim

In [ ]:
df.corr()

Isı Haritasını çıkaralım.

In [ ]:
corr= df.corr()
sns.heatmap(corr,xticklabels=corr.columns.values,yticklabels=corr.columns.values);

Fiyat aralığının benzersiz değerleri gösterelim.

In [ ]:
df["FiyatAraligi"].unique()

Fiyat Aralığının Benzersiz  olmayanların sayısını gösterelim.


In [ ]:
df["FiyatAraligi"].nunique()

Countplot kullanarak değişkenlerin sayılarına göre grafik çizdirelim.

In [ ]:
sns.countplot(df["FiyatAraligi"])

Veri setimizi Scatterplot kullanarak yükseklik ve genişlik arasındaki bağlantıyı gösterelim.

In [ ]:
sns.scatterplot(x = "CozunurlukYükseklik", y = "CozunurlukGenislik", data = df, color="red");

Yukarda yaptığımız işlemi jointplot kullanarak gerçekleştirelim


In [ ]:
sns.jointplot(x = "CozunurlukYükseklik", y = "CozunurlukGenislik", data = df, color="red");

Veri setimizin değişkenlerinin birbirlerine etkilerine göz atalım.

In [ ]:
sns.scatterplot(df["Agirlik"],df["CekirdekSayisi"])

Ağırlık ile Çekirdek Sayısının arasında ilişki bulunmadığını gördük.

In [ ]:
sns.scatterplot(x= "Agirlik" , y="WiFi" , data=df);

Ağırlık ile Wifi arasında bir ilişki bulunmadığını gördük.


In [ ]:
sns.scatterplot(df["CekirdekSayisi"],df["MikroislemciHizi"])

In [ ]:
sns.scatterplot(x = "OnKameraMP", y = "ArkaKameraMP", data = df);

In [ ]:
sns.jointplot(x = "OnKameraMP", y = "ArkaKameraMP", data = df, color="red");

Yukarıda aralarında ilişkilerini incelediğimiz verilerin keman grafiğini çizdirelim.

In [ ]:
sns.violinplot(y = "Kalinlik", data = df, color="blue");

In [ ]:
sns.violinplot(y = "MikroislemciHizi", data = df, color="blue");

In [ ]:
sns.violinplot(x=df["Agirlik"], color="blue");

Keman Grafiği ile Fiyat aralığı ve RAM arasındaki ilişkiyi gösterelim.

In [ ]:
sns.violinplot(df["FiyatAraligi"],df["RAM"])

Ön Kamera Mp değerine bağlı olarak Fiyat aralığının nasıl olduğunu inceleyelim.


In [ ]:
sns.scatterplot(x = "FiyatAraligi" , y ="OnKameraMP" , data = df);

RAM değerine bağlı olarak Fiyat aralığının nasıl olduğunu inceleyelim.


In [ ]:
sns.scatterplot(x = "RAM" , y ="OnKameraMP" , data = df);

RAM ile Batarya Ömrünü barplot kullanarak inceleyelim.

In [ ]:
sns.barplot(x ="BataryaOmru" , y = "RAM" , data = df);

Veri setimizin Histogram tablosunu çizdirelim.

In [ ]:
df.hist(figsize =(20,10), color = "pink")
plt.show()

# **Veri Ön İşlemeye Başlıyalım.**

Veri setimizin 20 gözlemini hatırlamak için tekrar görelim.

In [ ]:
df.head(20)

Eksik verilerimizin nerede olduğunu ve kaç tane olduğunu tekrardan bulalım.


In [ ]:
df.isnull().sum()

Isı haritasi çizdirelim.

In [ ]:
import missingno
missingno.heatmap(df,figsize = (8,8))

Eksik değerlerimizi görselleştirelim.

In [ ]:
missingno.matrix(df,figsize=(20, 10));

Eksik değerlerimizi doldurmak için öncelikle hangi fiyat aralıklarında olduklarına bakalım.

In [ ]:
df[df["OnKameraMP"].isnull()]

In [ ]:
df[df["RAM"].isnull()]

**Gördüğümüz üzere ön kamerası eksik olan tabletlerin hepsi çok ucuz kategorisinde, ramler ise pahalı kategorisinde bulunmakta.
İlk başta ön kamerası eksik olanları doldurmaya başlayalım.**

İlk başta boş olanları bir değişkene atayalım.

In [ ]:
okb= df[df["OnKameraMP"].isnull()]
okb

Şimdi atama yaptığımız değişkenden ucuz olmasının sebeplerine birkaç örnekle bakalım.

In [ ]:
print("Ortlama Mikro İşlemci Hızı:", df["MikroislemciHizi"].mean(), "\nEksiklerin Ortalama Mikro İşlemci Hızı:", okb["MikroislemciHizi"].mean())

Eksiklerin çok ucuz kategorisinde olmasının sebebinin mikro işlemci hızı olmadığını görmüş olduk.


In [ ]:
print("Ortlama Ram:", df["RAM"].mean(), "\nEksiklerin Ortalama RAM'i:", okb["RAM"].mean())


Çok ucuz kategorisinde bulunmalarının bir sebebi RAM olduğunu görmüş olduk.

In [ ]:
uoko=df[df["FiyatAraligi"]=="Çok Ucuz"][["OnKameraMP"]].mean()
uoko

Çok ucuz kategorisindeki ön kameraların ortalama değerlerini bir değişkene atadık ve bu değişkeni de boş olan indislere atayalım.

In [ ]:
okb=okb.fillna(uoko)
df[df["OnKameraMP"].isnull()] = okb
okb

**Görmüş olduğunuz gibi boş olan ön kamera bilgilerini çok ucuz kategorisindeki ortalama değere göre atamasını yapmış bulunmaktayız. Şimdi yukarıda yaptıklarımızın aynısını RAM içinde yapalım.**

In [ ]:
rb= df[df["RAM"].isnull()]
rb

Hepsinin pahalı kategorisinde olduğunu gördüğümüze göre pahalı kategorisindeki ortalama ram miktarını atama yapalım.

In [ ]:
rbo=df[df["FiyatAraligi"]=="Pahalı"][["RAM"]].mean()
rbo

In [ ]:
rb=rb.fillna(rbo)
df[df["RAM"].isnull()]=rb
rb

Artık boş olan değerlerimizin hepsini doldurmuş bulunmaktayız.

In [ ]:
missingno.matrix(df,figsize=(20, 10));

İlk başta renkleri sıralayabilmemiz için get_dummies fonksiyonunu kullanalım.

In [ ]:
df["Renk"] = pd.Categorical(df["Renk"])
df_Dummies = pd.get_dummies(df["Renk"])
df_Dummies

Şimdi renk kategorisinin dummies'ini aldığımıza göre artık renk kategorsiyle beraber bir tane rengi de çıkarabiliriz.


In [ ]:
df = pd.concat([df, df_Dummies], axis=1)
df.drop(["Kırmızı"], axis = 1, inplace = True)

Şimdi ise string verilerimizi integeri verilere dönüştürelim.

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
df.head()

Düzenlemeleri bitirdiğimize göre artık sıradaki işleme geçiyoruz.


# Makine öğrenmesine geçiyoruz.


Verilerimizi bağımlı ve bağımsız değişkenler olarak atama yapalım.

In [ ]:
X = df.drop("FiyatAraligi", axis = 1)
Y = df["FiyatAraligi"]

In [ ]:
Y

In [ ]:
X

Eğitme işlemimize başlıyoruz.

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
Y_train

In [ ]:
Y_test

In [ ]:
gnb = GaussianNB()
gnb_model = gnb.fit(X_train,Y_train)

X_test'i, Y'e tahmin ettirelim.

In [ ]:
Y_pred = gnb_model.predict(X_test)
Y_pred

Y_pred'in tahmininin tutarlılığına bakalım.

In [ ]:
gnb.score(X_train,Y_train)

In [ ]:
Tutarlilik = accuracy_score(Y_test,Y_pred)
Tutarlilik

Şimdi eğitimimiz için karmaşıklık matrisimizi çizdirelim.

In [ ]:
Karmasiklik_matrisi = confusion_matrix(Y_test, Y_pred)
Karmasiklik_matrisi

Şimdi ise hassasiyetimizi yazdıralım.

In [ ]:
Hassasiyet = precision_score(Y_test, Y_pred, average='weighted')
Hassasiyet

Şimdi ise bizden istenen diğer değerleri yazdıralım.

In [ ]:
Recall = recall_score(Y_test, Y_pred, average='weighted')
Recall

In [ ]:
F1 = f1_score(Y_test, Y_pred, average='weighted')  
F1

Şimdi de senaryoda istenilen  classification_report yazdıralım.


In [ ]:
print( classification_report(Y_test, Y_pred))

Sırada ise karar ağacı eğitimine başlayalım.

In [ ]:
dtc = DecisionTreeClassifier(random_state = 40, criterion='entropy')
dtc = dtc.fit(X_train, Y_train)

X_test'i, Y'e tahmin ettirelim.
​


In [ ]:
Y_pred = dtc.predict(X_test)
Y_pred

Y_pred'in tahmininin tutarlılığına bakalım.

In [ ]:
Tutarlilik = accuracy_score(Y_test,Y_pred)
Tutarlilik

Şimdi eğitimimiz için karmaşıklık matrisimizi çizdirelim.


In [ ]:
Karmasiklik_matrisi = confusion_matrix(Y_test, Y_pred)
Karmasiklik_matrisi

In [ ]:
cross_val_score(dtc, X, Y, cv = 10)

In [ ]:
cross_val_score(dtc, X, Y, cv = 10).mean()

In [ ]:
Hassasiyet = precision_score(Y_test, Y_pred, average='weighted')
Hassasiyet

In [ ]:
Recall = recall_score(Y_test, Y_pred, average='weighted')
Recall

In [ ]:
F1 = f1_score(Y_test, Y_pred, average='weighted')  
F1

In [ ]:
print(classification_report(Y_test, Y_pred))

Son olarak KNN modeli ile eğitimimizi yapıp kodumuzu bitiriyoruz.

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)

In [ ]:
Y_pred = knn.predict(X_test)

In [ ]:
Tutarlilik = accuracy_score(Y_test,Y_pred)
Tutarlilik

In [ ]:
Karmasiklik_matrisi = confusion_matrix(Y_test, Y_pred)
Karmasiklik_matrisi

In [ ]:
cross_val_score(knn, X, Y, cv = 10)

In [ ]:
Hassasiyet = precision_score(Y_test, Y_pred, average='weighted')
Hassasiyet

In [ ]:
Recall = recall_score(Y_test, Y_pred, average='weighted')
Recall

In [ ]:
F1 = f1_score(Y_test, Y_pred, average='weighted')  
F1

In [ ]:
print(classification_report(Y_test, Y_pred))

In [ ]:
score_list = []

for each in range(2,15,1):
    knn_classi = KNeighborsClassifier(n_neighbors = each)
    knn_classi.fit(X_train,Y_train)
    score_list.append(knn_classi.score(X_test, Y_test))

plt.plot(range(2,15,1),score_list)
plt.title("Komşu Sayısına Göre KNN Modelinin Doğruluk Skoru")
plt.xlabel("Komşu Miktarı")
plt.ylabel("Doğruluk skoru")
plt.show()